In [1]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tensorflow.keras import Model
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib

from IPython import display

In [10]:
# d1 = '/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2/client'
# d2 = '/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2/counselor'
# # iterate over files in
# # that directory

# data_in = []
# data_out = []
# for filename in sorted(os.listdir(d1)):
#     fn = os.path.join(d1, filename)
#     if os.path.isfile(fn):
#         with open(fn) as f:
#             lines = f.readlines()
#             aline = []
#             for x in lines:
#                 aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
#             data_in.append(aline)
#         f.close()


# for filename in sorted(os.listdir(d2)):
#     fn = os.path.join(d2, filename)
#     # checking if it is a file
#     if os.path.isfile(fn):
#         with open(fn) as f:
#             lines = f.readlines()
#             aline = []
#             for x in lines:
#                 aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
#             data_out.append(aline)
#         f.close()
        

In [11]:
# data_in = data_in[:len(data_out)]
# # print(tf.constant(data_in).shape)
# # print(tf.constant(data_out).shape)

# data = []
# sequence_len = 16
# ds = tf.data.Dataset.from_tensor_slices((tf.constant(data_in), tf.constant(data_out)))
# ds = ds.batch(sequence_len)
# ds = ds.shuffle(ds.cardinality().numpy())
# for inn, outt in ds:
#     print(inn.shape, outt.shape)

In [69]:
class GU(Model):

    def __init__(self, in_dim, num_channels=2, emb_dim=512, gate_filters=32, num_resolutions=3, attn_res_idx=1):
        super().__init__()
        
        self.in_dim = in_dim
        self.num_channels = num_channels
        self.num_resolutions = num_resolutions
        self.attn_res_idx = attn_res_idx
        self.emb_dim = emb_dim
        self.gate_filters = gate_filters
#         self.model = self.build_graph()
        
#     def timestep_embedding(self, ts, embedding_dim):
#         assert len(ts.shape)==1
        
#         half_dim = embedding_dim // 2
#         emb = -(tf.math.log(10000.0)/(half_dim-1))
        
#         idxs = tf.range(half_dim, dtype=tf.float32)
#         emb = tf.math.exp(idxs*emb)
        
#         ts = tf.cast(ts, dtype=tf.float32)
        
#         emb = ts[:,None]* emb[None,:]
        
#         pe = tf.concat([tf.sin(emb), tf.cos(emb)], axis=1)
        
#         return pe
    
    def nonlinearity(self, x):
        return tf.keras.activations.swish(x)
    
    def normalize(self, x):
        return tfa.layers.InstanceNormalization(axis=-1)(x)
        
    def ResBlK(self, x, temb=None):
        h = self.nonlinearity(self.normalize(x))
        h = layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(h)
        if not temb == None:
            temb = self.nonlinearity(temb)
            h +=  tf.reshape(layers.Dense(h.shape[-1])(temb), (-1, 1, 1,h.shape[-1]))

        h = self.nonlinearity(self.normalize(h))
        h = layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(h)
        
        return x+h
        
    def downsample(self, x, filters, factor=2):
        return layers.Conv2D(filters, (3, 3),
                                        strides=(factor, factor), padding='same')(x)

    def upsample(self, x, filters, factor=2):
        return layers.Conv2DTranspose(filters, (3, 3),
                                       strides=(factor, factor), padding='same')(x)
        
    def NonlocalGaussian(self, x):
        h = self.normalize(x)
        theta = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        g = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = tf.transpose(phi, (0, 3, 2, 1))

        theta = tf.reshape(theta, shape=(-1, theta.shape[1]*theta.shape[2], theta.shape[3]))

        phi = tf.reshape(phi, shape=(-1, phi.shape[1], phi.shape[2]*phi.shape[3]))
        g = tf.reshape(g, shape=(-1, g.shape[1]*g.shape[2], g.shape[3]))
        
        f = tf.matmul(theta, phi)
        
        f = layers.Softmax()(f)
        
        y = tf.matmul(f, g)
        
        y = tf.reshape(y, (-1, x.shape[1], x.shape[2], y.shape[-1]))
        
        z = tf.math.add(x, layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(y))
        return z
    
    def design(self, img_input):
        
        hs = [layers.Conv2D(self.gate_filters, (1, 1),
                                        strides=(1, 1), padding='same')(img_input)]
        
        for i in range(self.num_resolutions):
            h = self.ResBlK(hs[-1])
            if i == self.attn_res_idx:
                h = self.NonlocalGaussian(h)
                
            if i != self.num_resolutions - 1:
                h = self.downsample(h, h.shape[-1]*2)
                
            hs.append(h)

        h = hs[-1]
        h = self.ResBlK(h)
        h = self.NonlocalGaussian(h)
        h = self.ResBlK(h)
                
        for i in reversed(range(self.num_resolutions)):
            if i == self.attn_res_idx:
                h = self.NonlocalGaussian(h)   
            h = self.ResBlK(h)
            h = h+hs[i]
                
            if i != 0:
                h = self.upsample(h, h.shape[-1]//2)
                
        h = self.nonlinearity(self.normalize(h))
        
        h = layers.Conv2D(self.num_channels, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        
        return h
    
#     def call(self, x):
#         return self.model([x])
    
    def build_model(self):
        x = layers.Input(shape=(self.in_dim[0],
                                        self.in_dim[1], self.num_channels))
        
        return Model(inputs=[x], outputs=self.design(x))
        

In [3]:
unet1=GU((16, 68))

2023-03-26 21:37:20.214139: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-26 21:37:20.219129: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-26 21:37:20.219321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-26 21:37:20.219844: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
model1 = unet1.build_model()

In [48]:
# model1.summary()

In [34]:
# tf.keras.utils.plot_model(model1,to_file="./imgs/model.png", show_shapes=True, show_layer_activations=True)

In [6]:
# tf.keras.utils.model_to_dot(model1, show_shapes=True, show_layer_activations=True).write_raw("model.dot")
# graph.write_raw("output_raw.dot")

In [39]:
EPSILON = 1e-16

class model:
    def __init__(self, dataset_path, data_shape, num_channels, train_split=0.8,
                 batch_size=64, lr=3e-4):
        self.s = data_shape
        self.nc = num_channels
        self.bs = batch_size
        # self.dum = dum
        
        if not dataset_path==None:
            cl_path = dataset_path + "/client"
            co_path = dataset_path + "/counselor"

            self.ds = self.make_dataset(cl_path, co_path)
            
            ds_size = self.ds.cardinality().numpy()
            train_size = int(train_split * ds_size)
            # test_size = int((1.0-train_split) * ds_size)

            self.train_ds = self.ds.take(train_size)    
            self.test_ds = self.ds.skip(train_size)
        else:
            print("WARNING: Dataset not loaded, Model in Generator mode")
        
        self.g = GU(self.s, self.nc).build_model()
        self.g_opt = tf.keras.optimizers.Adam(lr)

    def make_dataset(self, d1, d2):
        data_in = []
        data_out = []
        for filename in sorted(os.listdir(d1)):
            fn = os.path.join(d1, filename)
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
                    data_in.append(aline)
                f.close()


        for filename in sorted(os.listdir(d2)):
            fn = os.path.join(d2, filename)
            # checking if it is a file
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
                    data_out.append(aline)
                f.close()

        data_in = data_in[:len(data_out)]
        # data = []
        sequence_len = self.s[0]
        ds = tf.data.Dataset.from_tensor_slices((tf.constant(data_in), tf.constant(data_out)))
        ds = ds.batch(sequence_len)
        ds = ds.shuffle(ds.cardinality().numpy())
        
        return ds
    
    def loss_func(self, pred_co, gt_co):
        return tf.reduce_mean(tf.math.sqrt(tf.reduce_sum(tf.math.add(pred_co, -tf.cast(gt_co, tf.float32))**2, axis = [1,2,3])+EPSILON))
        
    @tf.function
    def update(self, cl, co):
        with tf.GradientTape() as g_tape:
            pred_co = self.g(cl)
            g_loss = self.loss_func(pred_co, co)
            
        grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
        self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
        
        return g_loss
        
    def train(self, epochs=50):
        for epo in range(epochs):
            g_losses = []
            
            batch_inn = []
            batch_outt = []

            for inn, outt in self.train_ds:
                if inn.shape == (8,68,2) and outt.shape == (8,68,2):
                    if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                        tf_batch_inn = tf.stack(batch_inn)
                        tf_batch_outt = tf.stack(batch_outt)
                        g_losses.append(self.update(tf_batch_inn, tf_batch_outt))
                        batch_inn = []
                        batch_outt = []
                    else:
                        batch_inn.append(inn)
                        batch_outt.append(outt)


            print("Epoch {:04d}".format(epo), "Generator Avg. Loss: ", np.mean(g_losses), flush=True)
    def test(self):
        errors = []
        batch_inn = []
        batch_outt = []
        for inn, outt in self.test_ds:
            if inn.shape == (8,68,2) and outt.shape == (8,68,2):
                if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                    tf_batch_inn = tf.stack(batch_inn)
                    tf_batch_outt = tf.stack(batch_outt)
                    pred_co = self.g(tf_batch_inn)
                    errors.append(self.loss_func(pred_co, tf_batch_outt))
                    batch_inn = []
                    batch_outt = []
                else:
                    batch_inn.append(inn)
                    batch_outt.append(outt)
        print("Test Set Avg. Loss: ", np.mean(errors), flush=True)
        

In [40]:
ds_path = "/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2"

In [41]:
m1 = model(ds_path, (8,68), 2)

In [46]:
# m1.train(50)

In [47]:
# m1.test()

In [3]:
# batch_size=32
# batch_inn = []
# batch_outt = []


# for inn, outt in m1.ds:
#     if inn.shape == (8,68,2) and outt.shape == (8,68,2):
#         if (len(batch_inn))%batch_size ==0:
#             tf_batch_inn = tf.stack(batch_inn)
#             tf_batch_outt = tf.stack(batch_outt)
#             if not tf_batch_inn.shape[0] == 0:
#                 pred_out = m1.g(tf_batch_inn)
#                 print(pred_out.shape)
#             batch_inn = []
#             batch_outt = []
#             batch_inn.append(inn)
#             batch_outt.append(outt)
#         else:
#             batch_inn.append(inn)
#             batch_outt.append(outt)
#         print(batch_outt)


In [48]:
class AttnD(Model):
    
    def __init__(self, in_dim ,num_out_filter=1,
                  gate_filters=32):
#                  stage_filters=(256,256,256,256,256,256,256), stage_kernels=(3,3,3,3,3,3,3), stage_strides_ds=(2,2,2,2,2,2)):
        
        super().__init__()
        # staging params must be equal
        # assert len(stage_filters) == len(stage_kernels)
        # assert len(stage_filters) == len(stage_strides_ds)+1
        
        self.gate_filters = gate_filters
        self.in_dim = in_dim
#         self.attn_shape = attn_shape
        self.num_out_filter = num_out_filter

    def nonlinearity(self, x):
        return tf.keras.activations.swish(x)
    
    def normalize(self, x):
        return tfa.layers.InstanceNormalization(axis=-1)(x)
    
    def downsample(self, x, filters, factor=2):
        return layers.Conv2D(filters, (3, 3),
                                        strides=(factor, factor), padding='same')(x)
    
    def NonlocalGaussian(self, x):
        h = self.normalize(x)
        theta = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        g = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = tf.transpose(phi, (0, 3, 2, 1))

        theta = tf.reshape(theta, shape=(-1, theta.shape[1]*theta.shape[2], theta.shape[3]))

        phi = tf.reshape(phi, shape=(-1, phi.shape[1], phi.shape[2]*phi.shape[3]))
        g = tf.reshape(g, shape=(-1, g.shape[1]*g.shape[2], g.shape[3]))
        
        f = tf.matmul(theta, phi)
        
        f = layers.Softmax()(f)
        
        y = tf.matmul(f, g)
        
        y = tf.reshape(y, (-1, x.shape[1], x.shape[2], y.shape[-1]))
        
        z = tf.math.add(x, layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(y))
        return z

    def design(self, latent):
        
#         h = layers.Dense(self.attn_shape[0]*self.attn_shape[1]*self.gate_filters)(latent)
#         h = layers.Reshape((self.attn_shape[0], self.attn_shape[1], self.gate_filters))(h)
        h = layers.Conv2D(self.gate_filters, (1, 1),
                                        strides=(1, 1), padding='same')(latent)
        h = self.NonlocalGaussian(h)
        h=self.downsample(h,int(self.gate_filters*2.))
        h = self.NonlocalGaussian(h)
        h=self.downsample(h,int(self.gate_filters*2.))
        h = layers.Flatten()(h)
        h = layers.Dense(self.num_out_filter)(h)
        h = self.nonlinearity(h)
    
        return h

    def build_model(self):
        x = layers.Input(shape=self.in_dim)
        
        return Model(inputs=[x], outputs=self.design(x))

    
    # def build_graph(self):
    #     self.build(input_shape=(None,self.image_shape_in[0],
    #                                     self.image_shape_in[1], self.num_channel))
        
    # def reproduce_model(self):
    #     x = layers.Input(shape=(self.image_shape[0],
    #                                     self.image_shape[1], self.num_channel))
        
    #     return Model(inputs=x, outputs=self.call(x))
    
#     def plot_models(self,path):
#         rmodel = self.reproduce_model()
#         tf.keras.utils.plot_model(rmodel, to_file=path+"/model.png", show_shapes=True, show_layer_activations=True)
        
#         idx = 0
#         for resblk in self.resblks:
#             tf.keras.utils.plot_model(resblk.reproduce_model(self.res_inshape[idx], (1,1)), to_file=path+"/resblk_{:04d}.png".format(idx), show_shapes=True, show_layer_activations=True)
#             idx += 1


In [37]:
d = AttnD([16,68,2])

In [38]:
d1 = d.build_model()

In [41]:
# tf.keras.utils.plot_model(d1, to_file="./model_plots/model.png", show_shapes=True, show_layer_activations=True)

In [42]:
# d1.summary()

In [62]:
EPSILON = 1e-16

class gan_model:
    def __init__(self, dataset_path, data_shape, num_channels, dum=5, train_split=0.8,
                 batch_size=64, lr=3e-4, gp_lam=10.0):
        self.s = data_shape
        self.nc = num_channels
        self.bs = batch_size
        self.gp_lam = gp_lam
        self.dum = dum
        self.in_shape = list(self.s)
        self.in_shape.append(self.nc)
        
        if not dataset_path==None:
            cl_path = dataset_path + "/client"
            co_path = dataset_path + "/counselor"

            self.ds = self.make_dataset(cl_path, co_path)
            
            ds_size = self.ds.cardinality().numpy()
            train_size = int(train_split * ds_size)
            # test_size = int((1.0-train_split) * ds_size)

            self.train_ds = self.ds.take(train_size)    
            self.test_ds = self.ds.skip(train_size)
        else:
            print("WARNING: Dataset not loaded, Model in Generator mode")
        
        self.g = GU(self.s, self.nc).build_model()
        self.d = AttnD(self.in_shape).build_model()

        self.g_opt = tf.keras.optimizers.Adam(lr)
        self.d_opt = tf.keras.optimizers.Adam(lr)

    def make_dataset(self, d1, d2):
        data_in = []
        data_out = []
        for filename in sorted(os.listdir(d1)):
            fn = os.path.join(d1, filename)
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
                    data_in.append(aline)
                f.close()


        for filename in sorted(os.listdir(d2)):
            fn = os.path.join(d2, filename)
            # checking if it is a file
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        aline.append([int(y) for y in x.rstrip('\n').split(" ")[-2:]])
                    data_out.append(aline)
                f.close()

        data_in = data_in[:len(data_out)]
        # data = []
        sequence_len = self.s[0]
        ds = tf.data.Dataset.from_tensor_slices((tf.constant(data_in), tf.constant(data_out)))
        ds = ds.batch(sequence_len)
        ds = ds.shuffle(ds.cardinality().numpy())
        
        return ds
    
    def W_loss(self, d_data, d_gen, x_it):
        
        with tf.GradientTape() as t_gp:
            t_gp.watch(x_it)
            d_it = self.d(x_it)

        gp_grad = t_gp.gradient(d_it, x_it)
        l2n_gp = tf.math.sqrt(tf.reduce_sum(gp_grad**2, axis = [1,2,3])+EPSILON)
        L_gp = self.gp_lam*(l2n_gp-1.0)**2

        L_g = d_data - d_gen

        L_d = -L_g + L_gp

        return tf.nn.compute_average_loss(L_g), tf.nn.compute_average_loss(L_d)
    
    def loss_func(self, pred_co, gt_co):
        return tf.reduce_mean(tf.math.sqrt(tf.reduce_sum(tf.math.add(pred_co, -tf.cast(gt_co, tf.float32))**2, axis = [1,2,3])+EPSILON))
        
    # @tf.function
    # def update(self, cl, co):
    #     with tf.GradientTape() as g_tape:
    #         pred_co = self.g(cl)
    #         g_loss = self.loss_func(pred_co, co)
            
    #     grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
    #     self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
        
    #     return g_loss
    
    @tf.function 
    def update(self, cl, co, update_gen=True):
        
        # noise_input = tf.random.normal((imgs.shape[0], self.latent_dim))
            
        with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
            gco = self.g(cl)

            dx = self.d(co)
            dg = self.d(gco)

            epsi = tf.random.uniform([co.shape[0], 1, 1, 1], 0.0, 1.0)
            co_it = tf.math.add(epsi*tf.cast(co,tf.float32), (1.0-epsi)*gco)

            g_loss, d_loss = self.W_loss(dx, dg, co_it)

        if update_gen:
            grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
            grad_d = d_tape.gradient(d_loss, self.d.trainable_variables)

            self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
            self.d_opt.apply_gradients(zip(grad_d, self.d.trainable_variables))
        else:
            grad_d = d_tape.gradient(d_loss, self.d.trainable_variables)
            self.d_opt.apply_gradients(zip(grad_d, self.d.trainable_variables))

        return g_loss, d_loss

    def train(self, epochs=50):
        num_training = 0
        for epo in range(epochs):
            g_losses = []
            d_losses = []
            
            batch_inn = []
            batch_outt = []

            for inn, outt in self.train_ds:
                if list(inn.shape) == self.in_shape and list(outt.shape) == self.in_shape:
                    if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                        tf_batch_inn = tf.stack(batch_inn)
                        tf_batch_outt = tf.stack(batch_outt)
                        if num_training%self.dum == 0:
                            g_l, d_l = self.update(tf_batch_inn, tf_batch_outt, True)
                            g_losses.append(g_l.numpy())
                        else:
                            g_l, d_l = self.update(tf_batch_inn, tf_batch_outt, False)
                        d_losses.append(d_l.numpy())

                        num_training = (num_training+1)%self.dum
                        
                        batch_inn = []
                        batch_outt = []
                    else:
                        batch_inn.append(inn)
                        batch_outt.append(outt)


            print("Epoch {:04d}".format(epo), " Generator Avg. Loss: ", np.mean(g_losses),
                   ", Discriminator Avg. Loss: ", np.mean(d_losses), flush=True)
            
    def test(self):
        errors = []
        batch_inn = []
        batch_outt = []
        for inn, outt in self.test_ds:
            if inn.shape == (8,68,2) and outt.shape == (8,68,2):
                if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                    tf_batch_inn = tf.stack(batch_inn)
                    tf_batch_outt = tf.stack(batch_outt)
                    pred_co = self.g(tf_batch_inn)
                    errors.append(self.loss_func(pred_co, tf_batch_outt))
                    batch_inn = []
                    batch_outt = []
                else:
                    batch_inn.append(inn)
                    batch_outt.append(outt)
        print("Test Set Avg. Loss: ", np.mean(errors), flush=True)
        

In [63]:
ds_path = "/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2"

In [64]:
gan = gan_model(ds_path, (8,68), 2)

In [67]:
gan.train()

Epoch 0000  Generator Avg. Loss:  3371113.8 , Discriminator Avg. Loss:  -107966570.0
Epoch 0001  Generator Avg. Loss:  3352548.2 , Discriminator Avg. Loss:  -107638320.0
Epoch 0002  Generator Avg. Loss:  3352978.5 , Discriminator Avg. Loss:  -107548320.0
Epoch 0003  Generator Avg. Loss:  3353870.2 , Discriminator Avg. Loss:  -107264250.0
Epoch 0004  Generator Avg. Loss:  3352963.8 , Discriminator Avg. Loss:  -107280424.0
Epoch 0005  Generator Avg. Loss:  3348618.0 , Discriminator Avg. Loss:  -107357770.0
Epoch 0006  Generator Avg. Loss:  3342575.5 , Discriminator Avg. Loss:  -107152504.0
Epoch 0007  Generator Avg. Loss:  3325088.0 , Discriminator Avg. Loss:  -106934136.0
Epoch 0008  Generator Avg. Loss:  3334484.0 , Discriminator Avg. Loss:  -106777330.0
Epoch 0009  Generator Avg. Loss:  3329871.0 , Discriminator Avg. Loss:  -106654200.0
Epoch 0010  Generator Avg. Loss:  3316477.2 , Discriminator Avg. Loss:  -106553384.0
Epoch 0011  Generator Avg. Loss:  3312848.0 , Discriminator Avg. 

In [68]:
gan.test()

Test Set Avg. Loss:  7899.4136


In [10]:
class L2SAttn(layers.Layer):
    def __init__(self):
        super().__init__()

    def nonlinearity(self, x):
        return tf.keras.activations.swish(x)
    
    def normalize(self, x):
        return tfa.layers.InstanceNormalization(axis=-1)(x)

    def call(self, x):
        h = self.normalize(x)
        theta = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        g = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = tf.transpose(phi, (0, 3, 2, 1))

        theta = tf.reshape(theta, shape=(-1, theta.shape[1]*theta.shape[2], theta.shape[3]))

        phi = tf.reshape(phi, shape=(-1, phi.shape[1], phi.shape[2]*phi.shape[3]))
        g = tf.reshape(g, shape=(-1, g.shape[1]*g.shape[2], g.shape[3]))
        
        f = tf.matmul(theta, phi)
        
        f = layers.Softmax()(f)
        
        y = tf.matmul(f, g)
        
        y = tf.reshape(y, (-1, x.shape[1], x.shape[2], y.shape[-1]))
        
        z = tf.math.add(x, layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(y))
        return z
    

In [97]:
class L2SAttn_M(Model):

    def __init__(self, in_dim, num_channels=2):
        super().__init__()
        # self.l2sattn1 = L2SAttn()
        self.in_dim = in_dim
        self.nc = num_channels
        self.s = list(in_dim)
        self.s.append(self.nc)

    def nonlinearity(self, x):
        return tf.keras.activations.swish(x)
    
    def normalize(self, x):
        return tfa.layers.InstanceNormalization(axis=-1)(x)
    
    def call(self, x):
        h = self.normalize(x)
        Q = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        K = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        dK = tf.math.sqrt(tf.cast(K.shape[1]*K.shape[2]*K.shape[3], tf.float32))
        V = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        # phi = tf.transpose(phi, (0, 3, 2, 1))

        Q = tf.reshape(Q, shape=(-1, Q.shape[1]*Q.shape[2], Q.shape[3]))

        K = tf.reshape(K, shape=(-1, K.shape[1]*K.shape[2], K.shape[3]))
        V = tf.reshape(V, shape=(-1, V.shape[1]*V.shape[2], V.shape[3]))
        
        rnQ = tf.math.square(tf.math.reduce_euclidean_norm(Q, axis=-1, keepdims=True))
        f1 = tf.matmul(rnQ, tf.ones_like(rnQ), transpose_b=True)

        f2 = tf.matmul(Q, K, transpose_b=True)

        rnK = tf.math.square(tf.math.reduce_euclidean_norm(K, axis=-1, keepdims=True))
        f3 = tf.matmul(tf.ones_like(rnK), rnK, transpose_b=True)

        f = (f1-2*f2+f3)/dK
        
        f = layers.Softmax()(-f)
        
        y = tf.matmul(f, V)
        
        y = tf.reshape(y, (-1, x.shape[1], x.shape[2], y.shape[-1]))
        
        z = tf.math.add(x, layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(y))
        return z
    
    def build_graph(self):
        self.build(input_shape=(None,self.in_dim[0],
                                        self.in_dim[1], self.nc))

    def build_model(self):
        x = layers.Input(shape=self.s)
        
        return Model(inputs=[x], outputs=self.call(x))

In [98]:
a = L2SAttn_M((32,32), 32)

In [99]:
a.build_graph()

In [96]:
# tf.keras.utils.plot_model(a.build_model(),to_file="./model_plots/model.png", show_shapes=True, show_layer_activations=True)

In [81]:
Q = tf.random.uniform(shape=(2, 64, 32))
K = tf.random.uniform(shape=(2, 64, 32))

rnQ = tf.math.square(tf.math.reduce_euclidean_norm(Q, axis=-1, keepdims=True))
f1 = tf.matmul(rnQ, tf.ones_like(rnQ), transpose_b=True)

f2 = tf.matmul(Q, K, transpose_b=True)

rnK = tf.math.square(tf.math.reduce_euclidean_norm(K, axis=-1, keepdims=True))
f3 = tf.matmul(tf.ones_like(rnK), rnK, transpose_b=True)
f1.shape, f2.shape, f3.shape


(TensorShape([2, 64, 64]), TensorShape([2, 64, 64]), TensorShape([2, 64, 64]))

In [78]:
# tf.ones(shape=(None,3))